<h5 class='prehead'>SA405 &middot; Advanced Math Programming &middot; Fall 2021 &middot; Curry and Lourenco</h5>

<h5 class='lesson'>Lesson 2b - Student Version</h5>

<h1 class='lesson_title'>Pyomo Review</h1>

## This lesson...

- Implement the abstract model of the IP from Lesson 1 (listed below) in Pyomo:
    - Load the data into Python data structures
    - Build a Pyomo model
    - Solve the model
    - Print the solution
- Dealing with warnings when re-adding model components
- Full solution

**Instructions:** Read everything.  Run each line of code as you go.  Complete code as directed.

## Abstract model

We will implement the following abstract IP model from Lesson 1.  Refer to Lesson 1 for the data.

Sets
* $I := $ set of items that might go into the backpack  ({rope, matches, tent, sleeping bag, hammock, granola bar})
* $Q := $ set of item qualities that must be restricted ({volume, weight})

Parameters
* $a_{i,q} := $ amount of quality $q$ per item $i$, for $i \in I$, $q \in Q$
* $b_q := $ maximum amount of quality $q$ allowed, for $q \in Q$

Decision variables
* $x_i := 0/1$ variable that indicates whether or not to pack item $i$, for $i \in I$

Model
* Maximize $\displaystyle z := \sum_{i \in I} u_i x_i$

* subject to

    (1) $\displaystyle \sum_{i \in I} a_{i,q} x_i \leq b_q$, for all $q \in Q$

    (2) $x_i \in \{0,1\}$, for all $i \in I$

where the objective function maximizes the usefulness of the packed items, constraint (1) enforces the volume and weight limits, and (2) imposes the binary requirement on the $x$ variables.


## Load data into Python data structures

It is very difficult to read code for complicated models if the same single letter names are used in the code as are used in the model.  Fortunately, the Python community has well-defined naming practices.

**Python naming conventions**
* It is good coding practice to use short, descriptive names rather than single letters or numbers for variables
* We use the **constants** naming convention for model data (**sets** and **parameters**):  
  * UPPER_CASE, with words separated by underscores 
  * spaces are not allowed in Python names
* For more about Python naming conventions, see [PEP 8 naming styles](https://realpython.com/python-pep8/#naming-styles) and this [short video](https://realpython.com/lessons/python-naming-conventions/)

We will enter the data for the model line-by-line, running each line as we go.  This will allow us to debug as we go, and make it easy to isolate bugs.

#### Add Sets
* $I := $ set of items that might go into the backpack
* $Q := $ set of item qualities that must be restricted

**To code sets:**
* Use a Python **list** for a model set, because we can iterate over lists
    * Use brackets \[ \] to indicate a list

In [1]:
ITEMS = ['rope','matches','tent',
         'sleeping_bag','hammock','granola_bar']

* We can use the flexibility of Jupyter notebooks to test code snippets to see if our code is behaving as expected.  For example...
  * Use `type()` to verify that ITEMS is a list  
  * Run `ITEMS` to see the set in memory

In [2]:
type(ITEMS)

list

In [3]:
ITEMS

['rope', 'matches', 'tent', 'sleeping_bag', 'hammock', 'granola_bar']

# * **To do:** Enter the set Q in the cell below. Use the name QUALITIES, and the names 'weight' and 'volume'.

In [32]:
# To do

#### Add Parameters
* $u_{i} := $ usefulness of item $i$, for $i \in I$
* $b_{q} := $ maximum amount of quality $q$ allowed, for $q \in Q$
* $a_{i,q} := $ amount of quality $q$ item $i$ has, for $i \in I$, $q \in Q$

**To code parameters indexed by one or more sets:**
* Use a Python **dictionary**
    * Use brackets \{ \} to indicate a dictionary
    * Each entry is a **key:value** pair
    * Keys come from index set(s)
      * If there is only one index set, the key is an element of the set
      * If there are two or more index sets, the keys are **tuples** (indicated by parentheses)
    * The values are the numeric values of the parameter data
* Again we use the **constants** naming convention:  
    * UPPER_CASE, with words separated by underscores

In [33]:
USEFULNESS = {'rope':1, 'matches':5, 'tent':7, 'sleeping_bag':6,'hammock':4.5,'granola_bar':8}

# * **To do:** Enter the $b_q$ parameter values below.  Use the name MAX_AMOUNT.  Remember that this parameter is indexed by qualities, rather than items.  Refer to Lesson 1 to get the data values.

In [34]:
# To do

* The $a_{i,q}$ parameter is doubly-indexed over $I$ (ITEMS) AND $Q$ (QUALITIES), so we must use tuples as keys.

In [35]:
AMOUNT = {('rope','volume'):2, ('rope','weight'):3,
          ('matches','volume'):0.01, ('matches','weight'):0.1,
          ('tent','volume'):3, ('tent','weight'):10,
          ('sleeping_bag','volume'):2, ('sleeping_bag','weight'):4,
          ('hammock','volume'):0.4, ('hammock','weight'):4,
          ('granola_bar','volume'):0.67, ('granola_bar','weight'):2}

*Note that the use of descriptive names reduces the need for comments.*

## Build a Pyomo model

#### Import Pyomo

* Import the Pyomo library as follows. (This line of code is often at the top of the file.)
* After this import statement, we can use `pyo.` to access objects in the Pyomo library.

In [36]:
import pyomo.environ as pyo

* If you got an error on the line of code above, you probably need to install Pyomo by doing the following:  
  -  Uncomment the code in the cell below (keep the "!") and run the cell.  
  -  Then rerun the import statement above. 
  -  (You can now recomment this cell. You shouldn't have to install pyomo again in any notebook.)

In [37]:
#!conda install -c conda-forge pyomo

#### Instantiate a Pyomo ConcreteModel() object

In [38]:
model = pyo.ConcreteModel()

#### Add decision variables

* $x_i := 0/1$ variable that indicates whether or not to pack item $i$, for $i \in I$

* The naming convention for Python **variables** is lower-case letters or words, with words separated by underscores.  
    * We could use a word for this variable in the model, but since it is the only variable, I chose to stick with 'x'.
    * Another good choice would be 'pack', since the variable indicates whether or not to pack each item
* The line below adds the Pyomo variable 'x' to the 'model' that is...
  - indexed over ITEMS
  - has a Boolean domain (takes only 0/1 values)

In [39]:
# Add variable x indexed by the list ITEMS
model.x = pyo.Var(ITEMS, domain=pyo.Boolean)

* Note that we must use `model.` every time we access model components (variables, objective function, constraints)
* If we wanted $x \geq 0$ (rather than $x \in \{0,1\}$) we would use the following code:  
  - `model.x = pyo.Var(ITEMS, domain=NonNegativeReals)`.

* Use the following code to take a look at the current state of the 'model'
  -  Notice that the 'x' variables indexed by the set 'ITEMS' are now in the model, along with their the bounds and domain.
  - This code is not a necessary part of the model.  This is just a handy tool for making sure the model is coming along as expected.

In [40]:
model.display()

Model unknown

  Variables:
    x : Size=6, Index=x_index
        Key          : Lower : Value : Upper : Fixed : Stale : Domain
         granola_bar :     0 :  None :     1 : False :  True : Boolean
             hammock :     0 :  None :     1 : False :  True : Boolean
             matches :     0 :  None :     1 : False :  True : Boolean
                rope :     0 :  None :     1 : False :  True : Boolean
        sleeping_bag :     0 :  None :     1 : False :  True : Boolean
                tent :     0 :  None :     1 : False :  True : Boolean

  Objectives:
    None

  Constraints:
    None


#### Add the objective function

* Maximize $\displaystyle z := \sum_{i \in I} u_i x_i$

* The objective function is added in two steps:
  1. Define a small function using the keyword `def` that constructs the algebraic expression
  2. Add the objective to the model object using the small function

In [41]:
# Add objective function
def obj_rule(model):
    return sum(USEFULNESS[item]*model.x[item] for item in ITEMS)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

* Carefully compare the `return` statement of the `obj_rule` Python function with the objective function in the IP model
  - Notice how the summation over the list of ITEMS is constructed in Python using `sum()`
  

# * **TO ANSWER:**  What is the index variable in the Python code that plays the role of $i$ in the model constraint?
    (Enter answers as comments)

In [42]:
# To answer

* Now focus on the last line above:  `model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)`  
  - The `rule` keyword refers to the function to get the expression  
  - Notice we use `sense=pyo.maximize`, because we are maximizing the usefulness of the packed items

* Use the following code to take a look at the objective function.  Does it look like we expect?

In [43]:
# View the objective function
print(model.obj.expr)

x[rope] + 5*x[matches] + 7*x[tent] + 6*x[sleeping_bag] + 4.5*x[hammock] + 8*x[granola_bar]


#### Add constraints

(1) $\displaystyle \sum_{i \in I} a_{i,q} x_i \leq b_q$, for all $q \in Q$

*(Constriant (2) was taken care of when variable $x$ was added to the model)*

* The algebraic expression for constraint (1) has a summation over the ITEMS in it
  * (We are NOT summing over QUALITIES, instead there is a constraint for each quality (weight and volume))
  * Notice that we don't need to use `model.` to access the AMOUNT parameter because it is simply a Python object, not a 'model' component, like the variable 'x'

In [44]:
# Knapsack constraint function
def knapsack_rule(model, quality):
    return sum(AMOUNT[item,quality]*model.x[item] for item in ITEMS) <= MAX_AMOUNT[quality]

* Now look at the function `knapsack_rule` above
  - 'model' will always be the first parameter in a constraint (or objective) helper function
  - other parameters are **constraint index variables**
    * Here, there is one constraint for each 'quality' in 'QUALITIES', so 'quality' is the constraint index variable
* When we add the constraint to the model (below), 'QUALITIES' is the set the constraint is indexed over (i.e., there is one constraint per 'quality' in 'QUALITIES'

# * **TO ANSWER:**  Add the knapsack constraint to the model.

In [45]:
# Add knapsack constraint to the model (using the function above)

* Study this code for a while to make sure you see the difference between the: 
  -  **summation** index variable and index set (item, ITEMS) 
  and the 
  -  **constraint** index variable and index set (quality, QUALITIES)
* ***This is often the trickiest part of modeling with Pyomo***

* Use the following code to view the constraints we just added to the model
* Recall that the knapsack constraints are indexed by QUALITIES, so we can print them individually...

In [46]:
print(model.knapsack_constraint['weight'].expr)
print(model.knapsack_constraint['volume'].expr)

3*x[rope] + 0.1*x[matches] + 10*x[tent] + 4*x[sleeping_bag] + 4*x[hammock] + 2*x[granola_bar]  <=  12.5
2*x[rope] + 0.01*x[matches] + 3*x[tent] + 2*x[sleeping_bag] + 0.4*x[hammock] + 0.67*x[granola_bar]  <=  5.3



* Or we can iterate over QUALITIES to print them

In [47]:
for quality in QUALITIES:
    print(model.knapsack_constraint[quality].expr)

3*x[rope] + 0.1*x[matches] + 10*x[tent] + 4*x[sleeping_bag] + 4*x[hammock] + 2*x[granola_bar]  <=  12.5
2*x[rope] + 0.01*x[matches] + 3*x[tent] + 2*x[sleeping_bag] + 0.4*x[hammock] + 0.67*x[granola_bar]  <=  5.3


## Solve the model

*  *Note about solvers:* We are using the solver **glpk** for this course because it is a free, stand-alone solver that is easy to keep on your computer.  If you use optimization for your capstone project, you may need access to a more powerful solver.  To use a different solver that is installed on your computer, you just need to change 'glpk' to the apporpriate name (such as 'gurobi' or 'cplex').

* The following code sends the model to a solver that is installed on your computer 
  - `solver_result` contains some information from the solver (but not the solution!)

In [48]:
# solve model and save the result
solver_result = pyo.SolverFactory('glpk').solve(model)

* We can use `solver_result` to see if the solver found an optimal solution:

In [49]:
#Check how the solver terminated
print(solver_result.solver.termination_condition)

optimal


* One way to view the optimal solution is to display the model again:

In [50]:
model.display()

Model unknown

  Variables:
    x : Size=6, Index=x_index
        Key          : Lower : Value : Upper : Fixed : Stale : Domain
         granola_bar :     0 :   1.0 :     1 : False : False : Boolean
             hammock :     0 :   1.0 :     1 : False : False : Boolean
             matches :     0 :   1.0 :     1 : False : False : Boolean
                rope :     0 :   0.0 :     1 : False : False : Boolean
        sleeping_bag :     0 :   1.0 :     1 : False : False : Boolean
                tent :     0 :   0.0 :     1 : False : False : Boolean

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  23.5

  Constraints:
    knapsack_constraint : Size=2
        Key    : Lower : Body               : Upper
        volume :  None : 3.0799999999999996 :   5.3
        weight :  None :               10.1 :  12.5


* Notice that the variables have values now! 
* We can see the left-side value of each constraint with these variable values

# * **To answer:** According to the solution, which items should we pack to maximize usefulness?  

In [51]:
# To answer

# * **To answer:** What is the combined usefulness of the packed items?

In [52]:
# To answer

# * **To answer:** What is the combined weight and volume of the packed items?

In [53]:
# To answer


## Print the solution 

* Of course, for bigger models, it is difficult to view the model to find the optimal solution, so it is often a good idea to print the solution.
* **Method 1:** Loop over ITEMS and print the item name and the optimal value of the `x[item]`
  - To review **f-strings** for printing, see https://realpython.com/python-f-strings

In [54]:
print(f"The items to pack have combined usefulness rating {model.obj()}\n")
for item in ITEMS:
    print(f"x_{item} = {model.x[item].value}")

The items to pack have combined usefulness rating 23.5

x_rope = 0.0
x_matches = 1.0
x_tent = 0.0
x_sleeping_bag = 1.0
x_hammock = 1.0
x_granola_bar = 1.0


* **Method 2:** In this method we interpret the solution, rather than just printing it.  
  - We print only items that have x-values equal to 1

In [55]:
print(f'The items to pack have combined usefulness rating {model.obj()}\n')

print(f'Bring the following items:')
for item in ITEMS:
    if model.x[item].value == 1:
        print(f"{item}")

The items to pack have combined usefulness rating 23.5

Bring the following items:
matches
sleeping_bag
hammock
granola_bar


* Notice the weirdness in how the objective value is accessed: `model.obj()`, versus the variable values: `model.x[item].value`

## Dealing with warnings when re-adding model components

**If you rerun code to add a model component,** you will get a warning. 
* Try rerunning the cell that adds the knapsack_constraint to the model (below).  You should see a warning. 

In [56]:
# Add knapsack constraint to the model (using the function above)
model.knapsack_constraint = pyo.Constraint(QUALITIES, rule=knapsack_rule)

    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


RuntimeError: Cannot add component 'knapsack_constraint_index' (type <class 'pyomo.core.base.sets.SimpleSet'>) to block 'unknown': a component by that name (type <class 'pyomo.core.base.sets.SimpleSet'>) is already defined.

* Run the code below to delete the constraint.
* Run the cell that adds the knapsack_constraint (above) again.  You should see no warning this time.

In [57]:
#This code deletes the knapsack_constraint from the model
model.del_component('knapsack_constraint')

#If the constraint is indexed, we need this line, too
#This line deletes a 'hidden' index that was created behind the scenes
model.del_component('knapsack_constraint_index')

## Full solution: data, model, solve, print

* This is the full code to build and solve the model, with code grouped into cells in a useful way:
  - Import Pyomo
  - Construct data (sets and parameters)
  - Build model (decision variables, objective function, constraints)
  - Solve the model
  - Print solution
* In particular, soon we will define a Python function to build the model
* **However, when we build a model for the first time, keep the code separated into different cells.**
  - this makes debugging much easier

In [58]:
#Import pyomo
import pyomo.environ as pyo

In [59]:
#Construct data

#Sets
ITEMS = ['rope', 'matches', 'tent', 'sleeping_bag',
     'hammock','granola_bar']
QUALITIES = ['volume', 'weight']

#Parameters
USEFULNESS = {'rope':1, 'matches':5, 'tent':7, 
              'sleeping_bag':6, 'hammock':4.5,'granola_bar':8}

MAX_AMOUNT = {'volume':5.3, 'weight':12.5}

AMOUNT = {('rope','volume'):2, ('rope','weight'):3,
          ('matches','volume'):0.01, ('matches','weight'):0.1,
          ('tent','volume'):3, ('tent','weight'):10,
          ('sleeping_bag','volume'):2, ('sleeping_bag','weight'):4,
          ('hammock','volume'):0.4, ('hammock','weight'):4,
          ('granola_bar','volume'):0.67, ('granola_bar','weight'):2}

In [60]:
#Build model
model = pyo.ConcreteModel()

#Add decision variables
model.x = pyo.Var(ITEMS, domain=pyo.Boolean)

#Add objective function
def obj_rule(model):
    return sum(USEFULNESS[item]*model.x[item] for item in ITEMS)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

#Add knapsack constraint
#model.del_component(model.knapsack_constraint)
def knapsack_rule(model, quality):
    return sum(AMOUNT[item,quality]*model.x[item] for item in ITEMS) <= MAX_AMOUNT[quality]
model.knapsack_constraint = pyo.Constraint(QUALITIES, rule=knapsack_rule)

In [61]:
#Solve model
solver_result = pyo.SolverFactory('glpk').solve(model)
#Check if the solver found an optimal solution
print(solver_result.solver.termination_condition)

#or...

#Solve model, displaying solver output
#solver_result = pyo.SolverFactory('glpk').solve(model, tee=True)

optimal


In [62]:
#Print solution (if optimal solution was found)
print(f'Bring these items:')
for item in ITEMS:
    if model.x[item] == 1:
        print(item)
print(f'\n')
print(f'Their combined usefulness is {model.obj()}')

Bring these items:
matches
sleeping_bag
hammock
granola_bar


Their combined usefulness is 23.5
